# Quick start

In [ ]:
import admix
import numpy as np
import matplotlib.pyplot as plt
from os.path import join

In [ ]:
dset_dir = admix.dataset.get_test_data_dir()

In [ ]:
dset_admix = admix.io.read_dataset(join(dset_dir, "toy-admix"))
dset_all = admix.io.read_dataset(join(dset_dir, "toy-all"))


In [ ]:
dset_all

In [ ]:
dset_all.indiv

In [ ]:
ref_pops = ["CEU", "YRI"]

In [ ]:
ref_dsets = [
    dset_all[:, (dset_all.indiv["Population"] == pop).values] for pop in ref_pops
]
est = admix.ancestry.lanc(sample_dset=dset_admix, ref_dsets=ref_dsets)
admix.plot.lanc(lanc=est)

In [ ]:
admix.plot.lanc(lanc=dset_admix.lanc.compute())

In [ ]:
# Now we simulate phenotype for these admixed individuals.
np.random.seed(1234)
sim = admix.simulate.quant_pheno(dset=dset_admix, hsq=0.5, n_causal=2, n_sim=2)
beta, pheno_g, pheno = sim["beta"], sim["pheno_g"], sim["pheno"]
print(beta.shape)  # (n_snp, n_anc, n_sim)
print(pheno_g.shape)  # (n_indiv, n_sim)
print(pheno.shape)  # (n_indiv, n_sim)

In [ ]:
sim_i = 1
sim_pheno = pheno[:, sim_i]
sim_beta = beta[:, :, sim_i]
df_assoc = admix.assoc.marginal(
    dset=dset_admix,
    pheno=sim_pheno,
    method="ATT",
    family="linear",
)

In [ ]:
df_assoc

In [ ]:
fig, axes = plt.subplots(nrows=2, figsize=(5, 3), dpi=150, sharex=True)
axes[0].scatter(np.arange(dset_admix.n_snp), sim_beta[:, 0], s=2)
axes[0].set_ylabel(r"$\beta$")

admix.plot.manhattan(df_assoc.P, ax=axes[1], s=2)
plt.tight_layout()
plt.show()